In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.image as mpimg
%matplotlib inline

In [2]:
def readLog(csv_file):
    log = pd.read_csv(csv_file, header=None)
    for i in range(3):
        log.iloc[:,i] = log.iloc[:,i].str.replace('/home/jianjian/Nutstore/Self-Driving/Project4', '..')
        
    log = np.array(log)
    return log

log = readLog('../driving_log.csv')

In [3]:
def createXY(log):
    X_train = []
    y_train = []
    for ind, file in enumerate(log[:,0]):
        X_train.append(mpimg.imread(file))
        y_train.append(log[ind, 3])
    return np.array(X_train), np.array(y_train)

X_all, y_all = createXY(log)

In [4]:
X_train, y_train = X_all.astype(np.float32), y_all.astype(np.int32)
# X_valid, y_valid = X_all.astype(np.float32), y_all.astype(np.int32)

from keras.models import Sequential
from keras.layers import Flatten, Dense, BatchNormalization, Dropout, Cropping2D, Lambda, Activation
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D

Using TensorFlow backend.


In [5]:
mean = tf.keras.backend.constant(np.mean(X_train, axis=0))
std = tf.keras.backend.constant(np.std(X_train, axis=0))

In [6]:
mean.shape, std.shape

(TensorShape([Dimension(160), Dimension(320), Dimension(3)]),
 TensorShape([Dimension(160), Dimension(320), Dimension(3)]))

In [7]:
model = Sequential()
model.add(Lambda(lambda x: (x-128)/128, input_shape=(160, 320, 3)))
model.add(Cropping2D(cropping=((72, 20), (0, 0))))
model.add(Convolution2D(24,(5,5),activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D())
model.add(Convolution2D(36,(5,5),activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D())
model.add(Convolution2D(48,(5,5),activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D())
model.add(Convolution2D(64,(3,3),activation="relu"))
model.add(BatchNormalization())
model.add(Convolution2D(64,(3,3),activation="relu"))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dense(100))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Dense(50))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Dense(10))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Dense(1, activation=None))

model.compile(loss='mse', optimizer='adam')
model.fit(X_train, y_train, validation_split=0.2, shuffle=True, epochs=3)

model.save('model.h5')

Train on 11419 samples, validate on 2855 samples
Epoch 1/3
11419/11419 [==============================] - 461s 40ms/step - loss: 0.0407 - val_loss: 0.0068
Epoch 2/3
11419/11419 [==============================] - 460s 40ms/step - loss: 0.0036 - val_loss: 0.0022
Epoch 3/3
11419/11419 [==============================] - 431s 38ms/step - loss: 0.0022 - val_loss: 0.0019
